In [1]:
import astropy as ast
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris, EarthLocation, GeocentricTrueEcliptic, get_body, SkyCoord
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [4]:
data = pd.read_csv('solar_eclipse_data.csv')

# Select for years between 1900AD and 2100AD to work within datetime constraints
years_to_keep = data['Calendar Date'].str.split().apply(lambda x: x[0]).apply(lambda x: int(x) >= 1900 and int(x) <= 2050)
data = data[years_to_keep]
data.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration
9280,9281,1900 May 28,14:53:56,-2,-1232,126,T,0.3943,1.0249,44.8N,46.5W,67,175,92,02m10s
9281,9282,1900 November 22,07:19:43,-2,-1226,131,A,-0.2245,0.9421,33.1S,64.8E,77,7,220,06m42s
9282,9283,1901 May 18,05:33:48,-1,-1220,136,T,-0.3626,1.0680,1.7S,98.4E,69,353,238,06m29s
9283,9284,1901 November 11,07:28:21,0,-1214,141,A,0.4758,0.9216,10.8N,68.9E,62,190,336,11m01s
9284,9285,1902 April 8,14:05:06,0,-1209,108,Pe,1.5024,0.0643,71.7N,142.4W,0,67,NaN,NaN


In [5]:
data['Datetime'] = data['Calendar Date'] + ' ' + data['Eclipse Time']
data['Datetime'] = pd.to_datetime(data['Datetime'])
data['ISO Date'] = data['Datetime'].apply(lambda x: x.isoformat())
data['ISO Date'] = data['ISO Date'].apply(lambda x: Time(x))
data.head()

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,Longitude,Sun Altitude,Sun Azimuth,Path Width (km),Central Duration,Datetime,ISO Date
9280,9281,1900 May 28,14:53:56,-2,-1232,126,T,0.3943,1.0249,44.8N,46.5W,67,175,92,02m10s,1900-05-28 14:53:56,1900-05-28T14:53:56.000
9281,9282,1900 November 22,07:19:43,-2,-1226,131,A,-0.2245,0.9421,33.1S,64.8E,77,7,220,06m42s,1900-11-22 07:19:43,1900-11-22T07:19:43.000
9282,9283,1901 May 18,05:33:48,-1,-1220,136,T,-0.3626,1.0680,1.7S,98.4E,69,353,238,06m29s,1901-05-18 05:33:48,1901-05-18T05:33:48.000
9283,9284,1901 November 11,07:28:21,0,-1214,141,A,0.4758,0.9216,10.8N,68.9E,62,190,336,11m01s,1901-11-11 07:28:21,1901-11-11T07:28:21.000
9284,9285,1902 April 8,14:05:06,0,-1209,108,Pe,1.5024,0.0643,71.7N,142.4W,0,67,NaN,NaN,1902-04-08 14:05:06,1902-04-08T14:05:06.000


In [6]:
# All of the celestial bodies to generate synthetic data on
BODY_NAMES = ['moon', 'sun']

# Range is taken from the eclipse dataset
times = data['ISO Date']

# Location is the Medicina Radio Observatory, located in Italy. Chosen for proximity to Greece
loc = EarthLocation.of_site('medicina')
rows = defaultdict(list)

In [63]:
def get_coordinates(body):
    # Takes a Skycoord object, returns (theta, phi) in (deg, deg)
    angles = [float(i) for i in body.to_string().split(' ')]
    
    phi = angles[0]
    theta = angles[1]
    return phi, theta

for time in times:
    print(time)
    bodies = []
    
    with solar_system_ephemeris.set('builtin'):
        for body_name in BODY_NAMES:
            bodies.append(get_body(body_name, time, loc))

    #rows['time'].append(time)
    rows['location'].append(str(loc))
    
    for body_name, body in zip(BODY_NAMES, bodies):
        theta, phi = get_coordinates(body)
        ecliptic = SkyCoord(theta, phi, frame='gcrs', unit=('deg', 'deg')).transform_to(GeocentricTrueEcliptic())
        coordinates = get_coordinates(ecliptic)
        coord_strings = ['theta', 'phi']
        
        for i in range(len(coord_strings)):
            c = coordinates[i]
            rows[body_name + '_' + coord_strings[i]].append(c)

for k in rows.keys():
    data[k] = rows[k]

data.to_csv('solar_eclipse_data_coords.csv', index=False)

1900-05-28T14:53:56.000
1900-11-22T07:19:43.000
1901-05-18T05:33:48.000
1901-11-11T07:28:21.000
1902-04-08T14:05:06.000
1902-05-07T22:34:16.000
1902-10-31T08:00:18.000
1903-03-29T01:35:23.000
1903-09-21T04:39:52.000
1904-03-17T05:40:44.000
1904-09-09T20:44:21.000
1905-03-06T05:12:26.000
1905-08-30T13:07:26.000
1906-02-23T07:43:20.000
1906-07-21T13:14:19.000
1906-08-20T01:12:50.000
1907-01-14T06:05:43.000
1907-07-10T15:24:32.000
1908-01-03T21:45:22.000
1908-06-28T16:29:51.000
1908-12-23T11:44:28.000
1909-06-17T23:18:38.000
1909-12-12T19:44:48.000
1910-05-09T05:42:13.000
1910-11-02T02:08:32.000
1911-04-28T22:27:22.000
1911-10-22T04:13:02.000
1912-04-17T11:34:22.000
1912-10-10T13:36:14.000
1913-04-06T17:33:07.000
1913-08-31T20:52:12.000
1913-09-30T04:45:49.000
1914-02-25T00:13:01.000
1914-08-21T12:34:27.000
1915-02-14T04:33:20.000
1915-08-10T22:52:25.000
1916-02-03T16:00:21.000
1916-07-30T02:06:10.000
1916-12-24T20:46:22.000
1917-01-23T07:28:31.000
1917-06-19T13:16:21.000
1917-07-19T02:42

2022-04-30T20:42:36.000
2022-10-25T11:01:20.000
2023-04-20T04:17:56.000
2023-10-14T18:00:41.000
2024-04-08T18:18:29.000
2024-10-02T18:46:13.000
2025-03-29T10:48:36.000
2025-09-21T19:43:04.000
2026-02-17T12:13:06.000
2026-08-12T17:47:06.000
2027-02-06T16:00:48.000
2027-08-02T10:07:50.000
2028-01-26T15:08:59.000
2028-07-22T02:56:40.000
2029-01-14T17:13:48.000
2029-06-12T04:06:13.000
2029-07-11T15:37:19.000
2029-12-05T15:03:58.000
2030-06-01T06:29:13.000
2030-11-25T06:51:37.000
2031-05-21T07:16:04.000
2031-11-14T21:07:31.000
2032-05-09T13:26:42.000
2032-11-03T05:34:13.000
2033-03-30T18:02:36.000
2033-09-23T13:54:31.000
2034-03-20T10:18:45.000
2034-09-12T16:19:28.000
2035-03-09T23:05:54.000
2035-09-02T01:56:46.000
2036-02-27T04:46:49.000
2036-07-23T10:32:06.000
2036-08-21T17:25:45.000
2037-01-16T09:48:55.000
2037-07-13T02:40:36.000
2038-01-05T13:47:11.000
2038-07-02T13:32:55.000
2038-12-26T01:00:10.000
2039-06-21T17:12:54.000
2039-12-15T16:23:46.000
2040-05-11T03:43:02.000
2040-11-04T19:09

In [8]:
data = pd.read_csv('solar_eclipse_coords_data.csv')
data.head()

# TODO: add "labels" for eclipse datapoints, and sample non-eclipse datapoints

,Catalog Number,Calendar Date,Eclipse Time,Delta T (s),Lunation Number,Saros Number,Eclipse Type,Gamma,Eclipse Magnitude,Latitude,...,Sun Azimuth,Path Width (km),Central Duration,Datetime,ISO Date,location,moon_theta,moon_phi,sun_theta,sun_phi
0,9281,1900 May 28,14:53:56,-2,-1232,126,T,0.3943,1.0249,44.8N,...,175,92,02m10s,1900-05-28 14:53:56,1900-05-28T14:53:56.000,"(4461340.48313723, 919588.07009129, 4449530.22...",67.5568,0.007682,68.1689,0.011563
1,9282,1900 November 22,07:19:43,-2,-1226,131,A,-0.2245,0.9421,33.1S,...,7,220,06m42s,1900-11-22 07:19:43,1900-11-22T07:19:43.000,"(4461340.48313723, 919588.07009129, 4449530.22...",241.6440,-0.817203,240.9370,-0.013417
2,9283,1901 May 18,05:33:48,-1,-1220,136,T,-0.3626,1.0680,1.7S,...,353,238,06m29s,1901-05-18 05:33:48,1901-05-18T05:33:48.000,"(4461340.48313723, 919588.07009129, 4449530.22...",58.4661,-1.154280,57.9427,0.009681
3,9284,1901 November 11,07:28:21,0,-1214,141,A,0.4758,0.9216,10.8N,...,190,336,11m01s,1901-11-11 07:28:21,1901-11-11T07:28:21.000,"(4461340.48313723, 919588.07009129, 4449530.22...",230.2360,-0.132481,229.5880,-0.012132
4,9285,1902 April 8,14:05:06,0,-1209,108,Pe,1.5024,0.0643,71.7N,...,67,NaN,NaN,1902-04-08 14:05:06,1902-04-08T14:05:06.000,"(4461340.48313723, 919588.07009129, 4449530.22...",18.6202,1.111810,19.1625,0.004258
